<a href="https://colab.research.google.com/github/SerasLain/NLP-2019/blob/master/NLP_HW1_Sidorova.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашняя работа 1: разделение текста на предложения
## Анастасия Сидорова, БКЛ162

In [0]:
import re
import pandas as pd

In [0]:
# Let's get our sample!

def get_file(path):
  with open(path) as f:
      bare_text = f.read()
  return bare_text

class Text():
  def __init__(self):
    self.bare_text = get_file('./Что делать.txt')
    self.bare_sample = get_file('./sample_bare.txt')
    self.ideal_sample = [i.strip() for i in get_file('./sample_ideal.txt').strip().split('\n')]

In [0]:
class Sentencer():
  def simple_splitting(string):
    regex = '[.?!]\s[А-ЯЁ]'
    sentences = re.split(regex, string.strip())
    return sentences
  
  def evaluate(method, gold):
    tp = 0
    fp = 0
    fn = 0
    for sent in gold:
      if len(method(sent)) == 1:
          tp += 1
      else:
          fp += 1
    for i in range(len(gold)-1):
      sent1, sent2 = gold[i], gold[i+1]
      sent = ' '.join([sent1, sent2])
      if len(method(sent)) == 2:
          tp += 1
      else:
          fn += 1
    precision = (tp/(tp+fp))
    recall = (tp/(tp+fn))
    f1 = 2 * (precision * recall)/(precision + recall)
    print('Precision - ', precision)
    print('Recall - ', recall)
    print('F1 - ', f1)
    
    def __init__(self):
      self.sentences = []


In [0]:
t = Text()

In [106]:
# Metrics for simple splitting by '.?!'
Sentencer.evaluate(Sentencer.simple_splitting, t.ideal_sample)

Precision -  0.8636363636363636
Recall -  0.6627906976744186
F1 -  0.75


In [0]:
# Let's train PunktSentenceTokenizer!

from pprint import pprint
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktTrainer

def train_pst():
  trainer = PunktTrainer()
  trainer.INCLUDE_ALL_COLLOCS = True
  trainer.train(t.bare_text)
  tokenizer = PunktSentenceTokenizer(trainer.get_params())
  return tokenizer

tokenize = train_pst().tokenize

In [108]:
# Metrics for PunktSentenceTokenizer
Sentencer.evaluate(tokenize, t.ideal_sample)


Precision -  0.9259259259259259
Recall -  0.8426966292134831
F1 -  0.8823529411764706


In [109]:
# We've made some mistakes. Let's look through the list of sentences.

tokenized = set(tokenize(t.bare_sample)) - set(t.ideal_sample)

for i in tokenized:
  print(i[-15:])

о (т. XI, 1939.
 не возвращусь.
даже в тревоге.
овой идеологии.
я подготовил С.
ь лет спустя В.
я осуществил С.
 "Что делать?""
и "Что делать?"
 Б. Л. Кандель.
ической борьбы.
росил буфетчик.
та исправлений.
 3 часами ночи.
ан Афанасьевич?
А. Рейсер.
о "Что делать?"


In [0]:
# Adding abbreviaions
tokenizer._params.abbrev_types.add('с')
tokenizer._params.abbrev_types.add('в')
tokenizer._params.abbrev_types.add('1939')

In [113]:
# One more time
tokenized = set(tokenizer.tokenize(t.bare_sample)) - (set(t.ideal_sample))
for i in tokenized:
  print(i)

Полицейский чиновник подошел к столу, - на столе лежал лист бумаги, а на нем крупными буквами было написано: 

"Ухожу в 11 часов вечера и не возвращусь.
и примечания подготовил С. А. Рейсер; статью "Чернышевский-художник" - Г. Е. Тамарченко; черновой текст - Т. И. Орнатская; библиографию переводов на иностранные языки - Б. Л. Кандель.
Из рассказов о новых людях 

(Посвящается моему другу О.С.Ч.)  

I 

ДУРАК 

Поутру 11 июля 1856 года прислуга одной из больших петербургских гостиниц у станции московской железной дороги была в недоумении, отчасти даже в тревоге.
Основной текст романа, заметку для А. Н. Пыпина и Н. А. Некрасова, статью "Проблемы изучения романа "Что делать?""
Немало неточностей содержится и в издании "Что делать?"
в составе 16-томного "Полного собрания сочинений" Чернышевского (т. XI, 1939. Гослитиздат, подготовка Н. А. Алексеева и А. П. Скафтымова): по сравнению с ним в этой книге более ста исправлений.
был написан в стенах Петропавловской крепости в декабре 1862-апреле

In [114]:
# Metrics for PunktSentenceTokenizer after correcting
Sentencer.evaluate(tokenizer.tokenize, t.ideal_sample)

Precision -  0.9523809523809523
Recall -  0.8791208791208791
F1 -  0.9142857142857143


# Вывод

Кажется, что лучшего результата при помощи аббревиатур не добиться: часть ошибок произошла вследствие расхождения русской и английской пунктуации при оформлении заголовков и прямой речи. Как научить эту модель не реагировать на вопросительный знак перед кавычкой, мне непонятно.